In [1]:
import os
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
TESTTTTTTTTTTTTT

In [2]:
application_train = pd.read_csv('../../../homecredit/application_train.csv')
# POS_CASH_balance= pd.read_csv("../../../homecredit/POS_CASH_balance.csv")
previous_application = pd.read_csv("../../../homecredit/previous_application.csv")
# credit_card_balance= pd.read_csv("../../../homecredit/credit_card_balance.csv")
# installments_payments = pd.read_csv("../../../homecredit/installments_payments.csv")

In [8]:
previous_application=previous_application.sort_values(by=['SK_ID_CURR','SK_ID_PREV'])
previous_application.head(20)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
201668,1369693,100001,Consumer loans,3951.000,24835.50,23787.0,2520.00,24835.50,FRIDAY,13,...,Connectivity,8.0,high,POS mobile with interest,365243.0,-1709.0,-1499.0,-1619.0,-1612.0,0.0
892077,1038818,100002,Consumer loans,9251.775,179055.00,179055.0,0.00,179055.00,SATURDAY,9,...,Auto technology,24.0,low_normal,POS other with interest,365243.0,-565.0,125.0,-25.0,-17.0,0.0
575941,1810518,100003,Cash loans,98356.995,900000.00,1035882.0,NaN,900000.00,FRIDAY,12,...,XNA,12.0,low_normal,Cash X-Sell: low,365243.0,-716.0,-386.0,-536.0,-527.0,1.0
1223745,2396755,100003,Consumer loans,6737.310,68809.50,68053.5,6885.00,68809.50,SATURDAY,15,...,Consumer electronics,12.0,middle,POS household with interest,365243.0,-2310.0,-1980.0,-1980.0,-1976.0,1.0
1021650,2636178,100003,Consumer loans,64567.665,337500.00,348637.5,0.00,337500.00,SUNDAY,17,...,Furniture,6.0,middle,POS industry with interest,365243.0,-797.0,-647.0,-647.0,-639.0,0.0
935548,1564014,100004,Consumer loans,5357.250,24282.00,20106.0,4860.00,24282.00,FRIDAY,5,...,Connectivity,4.0,middle,POS mobile without interest,365243.0,-784.0,-694.0,-724.0,-714.0,0.0
1259112,1857999,100005,Cash loans,NaN,0.00,0.0,NaN,NaN,FRIDAY,10,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
1378978,2495675,100005,Consumer loans,4813.200,44617.50,40153.5,4464.00,44617.50,THURSDAY,11,...,Connectivity,12.0,high,POS mobile with interest,365243.0,-706.0,-376.0,-466.0,-460.0,0.0
1131133,1020698,100006,Cash loans,39954.510,454500.00,481495.5,NaN,454500.00,SATURDAY,12,...,XNA,18.0,high,Cash X-Sell: high,NaN,NaN,NaN,NaN,NaN,NaN
1232752,1243599,100006,Cash loans,NaN,0.00,0.0,NaN,NaN,THURSDAY,15,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
application_train = application_train[['SK_ID_CURR','AMT_ANNUITY','AMT_INCOME_TOTAL']]
application_train.sort_values(by=['SK_ID_CURR',])

,SK_ID_CURR,AMT_ANNUITY,AMT_INCOME_TOTAL
0,100002,24700.5,202500.0
1,100003,35698.5,270000.0
2,100004,6750.0,67500.0
3,100006,29686.5,135000.0
4,100007,21865.5,121500.0
...,...,...,...
307506,456251,27558.0,157500.0
307507,456252,12001.5,72000.0
307508,456253,29979.0,153000.0
307509,456254,20205.0,171000.0


In [11]:
#計算每個人過去總貸款每期應付款金額
prev_amt_annuity_sum = previous_application[["SK_ID_CURR","AMT_ANNUITY"]].groupby("SK_ID_CURR").sum().rename(columns={
            "AMT_ANNUITY": "prev_amt_annuity_sum"})
print(prev_amt_annuity_sum)
#合併到train
application_train = application_train[['SK_ID_CURR','AMT_ANNUITY','AMT_INCOME_TOTAL']]
Debt_Income_Ratio = application_train.merge(prev_amt_annuity_sum, on='SK_ID_CURR')
#將過去每期應付金額加上現在應附金額 / 每月收入 計算貸款收入率
Debt_Income_Ratio = (Debt_Income_Ratio.prev_amt_annuity_sum + Debt_Income_Ratio.AMT_ANNUITY)/Debt_Income_Ratio.AMT_INCOME_TOTAL
Debt_Income_Ratio = pd.DataFrame(Debt_Income_Ratio,columns=["Debt_Income_Ratio"])
Debt_Income_Ratio.info()
print(Debt_Income_Ratio.head(20))
#計算貸款收入率大於1的值有幾個 = 貸款金額比收入高的有幾人
Debt_Income_Ratio_Negative = (Debt_Income_Ratio.Debt_Income_Ratio > 1).sum()

print('----------------------------')
print(f"大於1的值有:{Debt_Income_Ratio_Negative}個")

            prev_amt_annuity_sum
SK_ID_CURR                      
100001                  3951.000
100002                  9251.775
100003                169661.970
100004                  5357.250
100005                  4813.200
...                          ...
456251                  6605.910
456252                 10074.465
456253                  9540.810
456254                 21362.265
456255                166203.135

[338857 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 291057 entries, 0 to 291056
Data columns (total 1 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Debt_Income_Ratio  291045 non-null  float64
dtypes: float64(1)
memory usage: 4.4 MB
    Debt_Income_Ratio
0            0.167666
1            0.760594
2            0.179367
3            1.271063
4            0.786324
5            0.917942
6            0.652988
7            0.193162
8            0.788765
9            0.325426
10   